In [5]:
import pandas as pd
data = pd.read_csv("C:/Users/lclai/Desktop/protein_fragmentation/data/aminoacids.csv")
data.head(5)

,id,aa,mf,energy,mw,polararea,complexity,xlogp,heavycnt,hbonddonor,hbondacc,rotbonds,exactmass,monoisotopicmass,covalentunitcnt,isotopeatomcnt,totalatomstereocnt,definedatomstereocnt,electrones,S
0,6274.0,hystidine,C6H9N3O2,-538.52442,155.15,92.0,151.0,-3.2,11.0,3.0,4.0,3.0,155.069,155.069,1.0,0.0,1.0,1.0,82.0,0.0
1,6306.0,leucine,C6H13NO2,-433.42225,131.17,63.3,103.0,-1.7,9.0,2.0,3.0,3.0,131.095,131.095,1.0,0.0,2.0,2.0,72.0,0.0
2,6106.0,isoleucine,C6H13NO2,-432.82708,131.17,63.3,101.0,-1.5,9.0,2.0,3.0,3.0,131.095,131.095,1.0,0.0,1.0,1.0,72.0,0.0
3,5962.0,lysine,C6H14N2O2,-487.74060,146.19,89.3,106.0,-3.0,10.0,3.0,4.0,5.0,146.106,146.106,1.0,0.0,1.0,1.0,80.0,0.0
4,6137.0,methionine,C5H11NO2S,-788.02138,149.21,88.6,97.0,-1.9,9.0,2.0,4.0,4.0,149.051,149.051,1.0,0.0,1.0,1.0,80.0,1.0


In [23]:
# reassembly for gly-val-ala
energy_water = -74.9659 #hartrees

#here filter to obtain your amino acid energies
energy_glycine = data.loc[data['aa'] == 'glycine', 'energy'].values[0]
energy_valine = data.loc[data['aa'] == 'valine', 'energy'].values[0]
energy_alanine = data.loc[data['aa'] == 'alanine', 'energy'].values[0]

reassembly_gly_val_ala = energy_glycine + energy_alanine + energy_valine - 2*energy_water

gt_gly_val_ala = -841.68835 #define here manually your ground truth (in our case computed through the ground_truth.py code)

print(f"Energy of gly-val-ala through reassembly is {reassembly_gly_val_ala} Hartrees")
print(f"Ground Truth gly-val-ala energy is {gt_gly_val_ala} Hartrees")

Energy of gly-val-ala through reassembly is -841.71855 Hartrees
Ground Truth gly-val-ala energy is -841.68835 Hartrees


In [30]:
#compute the RE% between the computed and the ground truth energies
import numpy as np

def relative_error(gt,computed):
    '''function to compute the relative percentage error between the reassembly and the grount
    truth energy, the arguments correpsond to this energies, respectively'''
    dif = np.abs(gt-computed)
    error = dif/np.abs(gt)
    percentage_error = error * 100
    return percentage_error
    
relative_error_gly_val_ala = relative_error(gt_gly_val_ala,reassembly_gly_val_ala)
print(f"The RE% of the reassebly for the gly-val-ala peptide is of {relative_error_gly_val_ala} %")

The RE% of the reassebly for the gly-val-ala peptide is of 0.003588026375800045 %
